# Golomb Ruler

This tutorial includes everything you need to set up decision optimization engines, build constraint programming models.

Table of contents:

-  [Describe the business problem](#Describe-the-business-problem)
*  [How decision optimization (prescriptive analytics) can help](#How--decision-optimization-can-help)
*  [Use decision optimization](#Use-decision-optimization)
    *  [Step 1: Model the Data](#Step-1:-Model-the-data)
    *  [Step 2: Set up the prescriptive model](#Step-2:-Set-up-the-prescriptive-model)
        * [Define the decision variables](#Define-the-decision-variables)
        * [Express the business constraints](#Express-the-business-constraints)
        * [Express the objective](#Express-the-objective)
        * [Solve with Decision Optimization solve service](#Solve-with-Decision-Optimization-solve-service)
    *  [Step 3: Investigate the solution and run an example analysis](#Step-3:-Investigate-the-solution-and-then-run-an-example-analysis)
*  [Summary](#Summary)
****

### Describe the business problem

* A detailed description (from which this paragraph comes from) is available on <b>Wikipedia</b> at <a href="https://en.wikipedia.org/wiki/Golomb_ruler" target="_blank" rel="noopener noreferrer">https://en.wikipedia.org/wiki/Golomb_ruler</a>.

* In mathematics, a Golomb ruler is a set of marks at integer positions along an imaginary ruler such that no two pairs of marks are the same distance apart. The number of marks on the ruler is its order, and the largest distance between two of its marks is its length. 

Following is an example of Golomb ruler of order 4 and length 6.
<center><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/0/05/Golomb_Ruler-4.svg/220px-Golomb_Ruler-4.svg.png"></center>
<p>
This problem is not only an intellectual problem. It has a lot of practical applications:
<ul>
<li> within Information Theory related to error correcting codes,
<li> the selection of radio frequencies to reduce the effects of intermodulation interference,
<li> the design of conference rooms, to maximize the number of possible configurations with a minimum of partitions:
</ul>
<center><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/5/52/Golomb_ruler_conference_room.svg/300px-Golomb_ruler_conference_room.svg.png"></center>


*****
## How  decision optimization can help
* Prescriptive analytics technology recommends actions based on desired outcomes, taking into account specific scenarios, resources, and knowledge of past and current events. This insight can help your organization make better decisions and have greater control of business outcomes.  

* Prescriptive analytics is the next step on the path to insight-based actions. It creates value through synergy with predictive analytics, which analyzes data to predict future outcomes.  

* Prescriptive analytics takes that insight to the next level by suggesting the optimal way to handle that future situation. Organizations that can act fast in dynamic conditions and make superior decisions in uncertain environments gain a strong competitive advantage.  
<br/>

+ For example:
    + Automate complex decisions and trade-offs to better manage limited resources.
    + Take advantage of a future opportunity or mitigate a future risk.
    + Proactively update recommendations based on changing events.
    + Meet operational goals, increase customer loyalty, prevent threats and fraud, and optimize business processes.


<h3>Modeling the problem</h3>
<p>
Constraint Programming is a programming paradigm that allows to express a problem using:

* the unknowns of the problem (the <i>variables</i>),
* the constraints/laws/rules of the problem, mathematical expressions linking variables together (the <i>constraints</i>),
* what is to be optimized (the <i>objective function</i>).

<p>
All this information, plus some configuration parameters, is aggregated into a single object called <i>model</i>. 
<p>
The remainder of this notebook describes in details how to build and solve this problem with IBM CP Optimizer, using its <i>DOcplex</i> Python modeling API.

## Use decision optimization

### Step 1: Model the data


In [1]:
# Import Constraint Programming modelization functions
from docplex.cp.model import CpoModel

<h4>Define model input data</h4>
<p>
The first thing to define is the model input data.
In the case of the Golomb Ruler problem, there is only one input which is the order of the ruler, that is the number of marks:

In [2]:
# Define required number of marks on the ruler
ORDER = 7

### Step 2: Set up the prescriptive model

<h4>Create the model container</h4>
<p>
The model is represented by a Python object that is filled with the different model elements (variables, constraints, objective function, etc). The first thing to do is then to create such an object:

In [3]:
# Create model object
mdl = CpoModel(name="GolombRuler")

#### Define the decision variables

* Now, you need to define the variables of the problem. As the expected problem result is the list of mark positions, the simplest choice is to create one integer variable to represent the position of each mark on the ruler.

* Each variable has a a set of possible values called his <i>domain</i>. To reduce the search space, it is important to reduce this domain as far as possible.

* In our case, we can naively estimate that the maximum distance between two adjacent marks is the order of the ruler minus one. Then the maximal position of a mark is (ORDER - 1)². Each variable domain is then limited to an interval [0..(ORDER - 1)²].

* A list of integer variables can be defined using method <i>integer_var_list()</i>. In our case, defining one variable for each mark can be created as follows:

In [4]:
# Create array of variables corresponding to ruler marks
marks = mdl.integer_var_list(ORDER, 0, (ORDER - 1) ** 2, "M")

#### Express the business constraints

* To express that all possible distances between two marks must be different, create an array that contains all these distances:

In [5]:
# Create an array with all distances between all marks
dist = [marks[i] - marks[j] for i in range(1, ORDER) for j in range(0, i)]

The operator '-' is used to express the difference between variables. This might appear strange as the variables are not instantiated at that time, but the Python operator has been overloaded to construct a CP expression instead of attempting to compute the arithmetic difference. All other standard Python operators can be used to make operations between CP objects (<, >, <=, >=, ==, !=, +, -, /, *, &, |, //, **,  ...). See documentation for details.
<p>
To force all these distances to be different, use the special <i>all_diff()</i> constraint as follows:

In [6]:
# Force all distances to be different
mdl.add(mdl.all_diff(dist))

The call <i>mdl.add(...)</i> is necessary to express that the constraint must be added to the model.

<h4>Remove symmetries</h4>
<p>
The constraint you have expressed above is theoretically sufficient, and the model can be solved as it is.
<p>
However, it does not differentiate between all possible permutations of the different mark positions that are solutions to the problem, for example, 0-1-4-6, 4-6-1-0, 6-0-1-4, etc. As there are ORDER! (factorial of ORDER) such permutations, the search space would be drastically reduced by removing them.
<p>
You can do that by forcing an order between marks, for example the order of their index:

In [7]:
# Avoid symmetric solutions by ordering marks
for i in range(1, ORDER):
    mdl.add(marks[i] > marks[i - 1])

You also know that first mark is at the beginning of the ruler:

In [8]:
# Force first mark position to zero
mdl.add(marks[0] == 0)

<h4>Avoid mirror solutions</h4>
<p>
Each optimal solution has a mirror, with all mark distances in the reverse order, for example, 0-1-4-6 and 0-2-5-6. 
The following constraint can be added to avoid this: 

In [9]:
# Avoid mirror solution
mdl.add((marks[1] - marks[0]) < (marks[ORDER - 1] - marks[ORDER - 2]))

#### Express the objective

* Finally, to get the shortest Golomb Ruler, this can be expressed by minimizing the position of the last mark.
As you have ordered the marks, you can do this using:

In [10]:
# Minimize ruler size
mdl.add(mdl.minimize(marks[ORDER - 1]))

If the marks were not ordered, you could have instead used:<br>
<code>   mdl.add(mdl.minimize(mdl.max(marks)))</code><br>

#### Solve with Decision Optimization solve service

By default, the modeling layer looks for a local runtime, but other solving environments, such as *docloud*, are also available.
Refer to the documentation for a good understanding of the various solving/generation modes.

If you're using a Community Edition of CPLEX runtimes, depending on the size of the problem, the solve stage might fail and will need a paying subscription or product installation.

The model can be solved by calling:

In [11]:
# Solve the model
print("Solving model....")
msol = mdl.solve(TimeLimit=10)

Solving model....


### Step 3: Investigate the solution and then run an example analysis

The shortest way to output the solution that has been found by the solver is to call the method <i>print_solution()</i> as follows:

In [12]:
# Print solution
print("Solution: ")
msol.write()

Solution: 
-------------------------------------------------------------------------------
Model constraints: 9, variables: integer: 7, interval: 0, sequence: 0
Solve status: Feasible, Fail status: SearchStoppedByLimit
Search status: SearchStopped, stop cause: SearchStoppedByLimit
Solve time: 10.4 sec
-------------------------------------------------------------------------------
Objective values: (25,), bounds: (20,), gaps: (0.2,)
M_0: 0
M_1: 1
M_2: 11
M_3: 16
M_4: 19
M_5: 23
M_6: 25


This output is totally generic and simply prints the value of all model variables, the objective value, and some other solution information.
<p>
A more specific output can be generated by writing more code. The following example illustrates how to access specific elements of the solution. 

In [13]:
# Print solution
from sys import stdout
if msol:
    # Print found solution
    stdout.write("Solution: " + msol.get_solve_status() + "\n")
    stdout.write("Position of ruler marks: ")
    for v in marks:
        stdout.write(" " + str(msol[v]))
    stdout.write("\n")
    stdout.write("Solve time: " + str(round(msol.get_solve_time(), 2)) + "s\n")
else:
    # No solution found
    stdout.write("No solution found. Search status: " + msol.get_solve_status() + "\n")

Solution: Feasible
Position of ruler marks:  0 1 11 16 19 23 25
Solve time: 10.4s


Another possibility is for example to simulate a real ruler using characters, as follows:

In [14]:
# Print solution as a ruler
if msol:
    stdout.write("Ruler: +")
    for i in range(1, ORDER):
        stdout.write('-' * (msol[marks[i]] - msol[marks[i - 1]] - 1) + '+')
    stdout.write("\n")

Ruler: ++---------+----+--+---+-+


<h3>Going further with Constraint Programming</h3>

The last available installable package is available on Pypi here: <a href="https://pypi.python.org/pypi/docplex" target="_blank" rel="noopener noreferrer">https://pypi.python.org/pypi/docplex</a>

A complete set of modeling examples can be downloaded here: <a href="https://github.com/IBMDecisionOptimization/docplex-examples" target="_blank" rel="noopener noreferrer">https://github.com/IBMDecisionOptimization/docplex-examples</a>

## Summary

You have learned how to set up and use the IBM Decision Optimization CPLEX Modeling for Python to build and solve a Constraint Programming model.

#### References
* <a href="https://rawgit.com/IBMDecisionOptimization/docplex-doc/master/docs/index.html" target="_blank" rel="noopener noreferrer">Decision Optimization CPLEX Modeling for Python documentation</a>
* <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/getting-started/welcome-main.html" target="_blank" rel="noopener noreferrer">Watson Studio documentation</a>


<hr>
Copyright © 2017-2019. This notebook and its source code are released under the terms of the MIT License.

<div style="background:#F5F7FA; height:110px; padding: 2em; font-size:14px;">
<span style="font-size:18px;color:#152935;">Love this notebook? </span>
<span style="font-size:15px;color:#152935;float:right;margin-right:40px;">Don't have an account yet?</span><br>
<span style="color:#5A6872;">Share it with your colleagues and help them discover the power of Watson Studio!</span>
<span style="border: 1px solid #3d70b2;padding:8px;float:right;margin-right:40px; color:#3d70b2;"><a href="https://www.ibm.com/products/watson-studio" target="_blank" style="color: #3d70b2;text-decoration: none;">Sign Up</a></span><br>
</div>
